# Primer agenta za rezervacijo hotela in letenke

Ta rešitev vam bo pomagala rezervirati letalske vozovnice in hotel. Scenarij je potovanje London Heathrow LHR 20. februar 2024 do New York JFK z vrnitvijo 27. februarja 2025, let v ekonomskem razredu z British Airways. Želim bivanje v hotelu Hilton v New Yorku, prosim, navedite stroške za let in hotel.


# Inicializirajte storitev Azure AI Agent in pridobite konfiguracijske informacije iz **.env**

### **.env** 

Ustvarite datoteko .env 

**.env** vsebuje povezovalni niz za storitev Azure AI Agent, model, ki ga uporablja AOAI, in ustrezne API-je ter ENDPOINT za Google API Search storitev.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Ime vašega modela za Azure AI Agent storitev"

[**NOTE**] Potrebovali boste model z 100.000 omejitvijo hitrosti (žetoni na minuto) in omejitvijo 600 (zahtev na minuto)

  Model lahko pridobite v Microsoft Foundry - Model in Endpoint. 


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Povezovalni niz za vaš projekt Azure AI Agent storitve"

  Povezovalni niz projekta lahko dobite v povzetku vašega projekta na zaslonu AI Foundry Portala.

- **SERPAPI_SEARCH_API_KEY** = "Vaš SERPAPI iskalni API ključ"
- **SERPAPI_SEARCH_ENDPOINT** = "Vaš SERPAPI iskalni Endpoint"

Za pridobitev imena modela za uvajanje in povezovalnega niza projekta Azure AI Agent storitve morate ustvariti Azure AI Agent Service. Priporočljivo je, da uporabite [to predlogo](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) za neposredno ustvarjanje (***Opomba:*** Storitev Azure AI Agent je trenutno na voljo samo v omejenih regijah. Priporočamo, da se sklicujete na [to povezavo](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) za nastavitev regije)

Agent mora imeti dostop do SERPAPI. Priporočljivo je, da se registrirate preko [te povezave](https://serpapi.com/searches). Po registraciji lahko pridobite edinstven API ključ in ENDPOINT.


# Namestitev

Za zagon tega zvezka morate namestiti potrebne knjižnice z ukazom `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Vaša različica Semantic Kernel mora biti vsaj 1.27.2.


Naložite svoje nastavitve in vire iz datoteke .env. Prosimo, zagotovite, da ste dodali svoje ključe in nastavitve ter ustvarili lokalno datoteko .env.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Prijava v Azure

Zdaj se morate prijaviti v Azure. Odprite terminal in zaženite naslednji ukaz:

```bash
az login
```

Ta ukaz vas bo pozval, da vnesete svoje Azure poverilnice, kar bo omogočilo pravilno delovanje storitve Azure AI Agent.


# Razlaga:
To je spremenljivka, ki shrani API ključ za dostop do SERP (Search Engine Results Page) API storitve. API ključ je edinstven identifikator, ki se uporablja za preverjanje pristnosti zahtevkov, povezanih z vašim računom.

Namen: Namen te vrstice je shraniti API ključ v spremenljivko, da se lahko uporabi za preverjanje pristnosti zahtevkov do SERP API storitve. Za dostop do storitve in izvajanje iskanj je potreben API ključ.
Kako pridobiti SERP API ključ: Če želite pridobiti SERP API ključ, sledite tem splošnim korakom na https://serpapi.com (natančni koraki se lahko razlikujejo glede na specifično SERP API storitev, ki jo uporabljate):

Izberite SERP API storitev: Na voljo je več SERP API storitev, kot so SerpAPI, Google Custom Search JSON API in druge. Izberite tisto, ki najbolj ustreza vašim potrebam.

Registrirajte se za račun: Obiščite spletno stran izbrane SERP API storitve in se registrirajte za račun. Morda boste morali podati nekaj osnovnih informacij in potrditi svoj e-poštni naslov.

Ustvarite API ključ: Po registraciji se prijavite v svoj račun in pojdite v razdelek API ali kontrolno ploščo. Poiščite možnost za ustvarjanje ali generiranje novega API ključa.
Kopirajte API ključ v vašo datoteko .env.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Razlaga:
BASE_URL: To je spremenljivka, ki shrani osnovni URL za SERP API končno točko. Ime spremenljivke BASE_URL je konvencija, ki se uporablja za označevanje, da je ta URL izhodišče za ustvarjanje API zahtevkov.  
'https://serpapi.com/search':

To je dejanska niz URL, dodeljen spremenljivki BASE_URL. Predstavlja končno točko za izvajanje iskalnih poizvedb z uporabo SERP API-ja.

# Namen:
Namen tega stavka je definirati konstantno vrednost, ki hrani osnovni URL za SERP API. Ta URL bo uporabljen kot izhodišče za sestavljanje API zahtev za izvajanje iskalnih operacij.

# Uporaba:
Z definiranjem osnovnega URL v spremenljivki ga lahko preprosto ponovno uporabite v celotni kodi, kadar je potrebno narediti zahtevke do SERP API-ja. To naredi vašo kodo bolj vzdrževalno in zmanjša tveganje napak zaradi trdega kodiranja URL-ja na več mestih. Trenutni primer je https://serpapi.com/search?engine=bing, ki uporablja Bing iskalni API. Drugi API lahko izberete na https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Razlaga:

Tukaj se nahaja vaša koda vtičnika.

Definicija razreda: `class BookingPlugin`: Določa razred z imenom BookingPlugin, ki vsebuje metode za rezervacijo hotelov in letov.

Metoda za rezervacijo hotela:

- `@kernel_function(description="booking hotel")`: Dekorator, ki opisuje funkcijo kot jedrno funkcijo za rezervacijo hotelov.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Določa metodo za rezervacijo hotelov z anotiranimi parametri in tipom vračanja.

Metoda sestavi slovar parametrov za zahtevo rezervacije hotela in pošlje GET zahtevo na SERP API. Preveri status odgovora in vrne podatke o hotelu, če je bila zahteva uspešna, sicer None.

Metoda za rezervacijo leta:

- `@kernel_function(description="booking flight")`: Dekorator, ki opisuje funkcijo kot jedrno funkcijo za rezervacijo letov.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Določa metodo za rezervacijo letov z anotiranimi parametri in tipom vračanja.

Metoda sestavi slovarje parametrov za odhodni in povratni let ter pošlje GET zahteve na SERP API. Preveri status odgovora in doda informacije o letu v rezultat, če je bila zahteva uspešna, sicer izpiše sporočilo o napaki. Metoda vrne rezultat, ki vsebuje podatke o letu.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Razlaga:
Izjave za uvoz: Uvozi potrebne module za Azure poverilnice, AI agenta, vsebino klepetalnih sporočil, vlogo avtorja in dekorator funkcije jedra.

Asinhroni upravljalec konteksta: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): To vzpostavi asinhroni upravljalec konteksta za upravljanje Azure poverilnic in ustvarjanje AI agentovega klienta.

Ime agenta in navodila:
- `AGENT_NAME = "BookingAgent"`: Določi ime agenta.
- `AGENT_INSTRUCTIONS = """..."""`: Zagotavlja podrobna navodila agentu, kako ravnati z zahtevami za rezervacijo.

Ustvari definicijo agenta: `agent_definition = await client.agents.create_agent(...)`: Ustvari definicijo agenta z določenim modelom, imenom in navodili.

Ustvari AzureAI agenta: `agent = AzureAIAgent(...)`: Ustvari AzureAI agenta z uporabo klienta, definicije agenta in določenega vtičnika.

Ustvari nit: `thread: AzureAIAgentThread | None = None`: Ustvari nit za agenta. Ni potrebno najprej ustvariti nita - če je podana vrednost `None`, bo nova nit ustvarjena med prvim klicem in vrnjena kot del odziva.

Uporabniški vnosi: `user_inputs = ["..."]`: Določi seznam uporabniških vhodov, ki jih mora agent obdelati.

V bloku finally izbriši nit in agenta za čiščenje virov.


# Overjanje

Razred `DefaultAzureCredential` je del Azure SDK za Python. Ponuja privzet način za overjanje z Azure storitvami. Poskuša se overiti z uporabo več metod v določenem vrstnem redu, kot so okoljske spremenljivke, upravljana identiteta in Azure CLI poverilnice.

Asinhrone operacije: modul aio pomeni, da razred DefaultAzureCredential podpira asinhrone operacije. To pomeni, da ga lahko uporabljate z asyncio za izvajanje neblokirnih zahtevkov za overjanje.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo AI prevajalske storitve [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, upoštevajte, da avtomatizirani prevodi lahko vsebujejo napake ali netočnosti. Izvirni dokument v izvirnem jeziku velja za avtoritativni vir. Za pomembne informacije priporočamo strokovni človeški prevod. Nismo odgovorni za morebitne nesporazume ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
